In [1]:
import pandas as pd
import xmltodict
import requests as req
import re

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy
from spacy import displacy
from spacy.lang.es import Spanish
from spacy_spanish_lemmatizer import SpacyCustomLemmatizer
from spacy.lang.es.stop_words import STOP_WORDS

# XML TO DIC

In [2]:
#parse xml to dic
def getxml():
    url = 'http://tass.sepln.org/tass_data/dataset/TASS2019_country_MX_train.xml'
    data = req.get(url).content
    return xmltodict.parse(data)

In [3]:
def getxml1():
    url = 'http://tass.sepln.org/tass_data/dataset/TASS2019_country_MX_dev.xml'
    data = req.get(url).content
    return xmltodict.parse(data)

In [4]:
def getxml2():
    url = 'http://tass.sepln.org/tass_data/dataset/general-train-tagged-3l.xml'
    data = req.get(url).content
    return xmltodict.parse(data)

In [5]:
OrderedDict = getxml()

orderedDictList = OrderedDict['tweets']['tweet']

#dic to dataframe
df = pd.DataFrame(orderedDictList)

#flatten sentiment column
df = pd.concat([df.sentiment.apply(pd.Series), df.drop('sentiment', axis=1)], axis=1)


In [6]:
OrderedDict1 = getxml1()

orderedDictList1 = OrderedDict1['tweets']['tweet']

#dic to dataframe
df1 = pd.DataFrame(orderedDictList1)

#flatten sentiment column
df1 = pd.concat([df1.sentiment.apply(pd.Series), df1.drop('sentiment', axis=1)], axis=1)

In [7]:
OrderedDict2 = getxml2()

orderedDictList2 = OrderedDict2['tweets']['tweet']

#dic to dataframe
df2 = pd.DataFrame(orderedDictList2)

#flatten sentiment column
df2 = pd.concat([df2.sentiments.apply(pd.Series), df2.drop('sentiments', axis=1)], axis=1)

In [8]:
df=df.append(df1)

In [10]:
def get_values(s):
    return s.values()

In [ ]:
df['polarity'].apply(lambda x: get_values(x))

In [12]:
#code values to -1, 0, 1
def code(s):
    if 'NEU' in s:
        s = .5
    elif 'N' in s:
        s = 0
    elif 'P' in s:
        s = 1
    elif 'NONE' in s:
        s = 'unknown'
    return s

In [13]:
df['polarity'] = df['polarity'].apply(lambda x: code(x))

In [45]:
df.to_csv('corpus_train.csv')

In [15]:
#drop tweets that don't have a polarity
df = df[df.polarity != 'unknown']

In [16]:
#select only useful columns for training
train_twitter = df[['content','polarity']]

In [17]:
train_twitter.reset_index(inplace=True)

In [18]:
train_twitter = train_twitter.drop(columns='index')

In [399]:
def lemmatize_(s):
    lst=[]
    s=nlp(s)
    for token in s:
        lst.append(token.lemma_)
    return lst

In [402]:
train_twitter['content'].apply(lambda x : lemmatize_(x))

0       [nivel, de, ingle,    , alto,    , traducir, j...
1       [si, ser, de, area,   , y, con, suerte, pasar,...
2       [sabian, que, su, after, fav, teatro, gramo,  ...
3       [y, hoy, por, primero, vez, me, sali, con, el,...
4       [parir, que, hacer, coraje, con, ambriz,   , a...
                              ...                        
1353               [jancarlobg, a, vez, me, sentir, solo]
1354    [tomar, en, miercoles,     , na,  , solo, olvi...
1355                 [querer, y, necesitar, verte, yaaaa]
1356    [gracia, a, comedy, central, y, mtv, ir, a, es...
1357    [por, si, no, saber,  , al, tirar, ese, a, lo,...
Name: content, Length: 1358, dtype: object

# CONTENT CLEANING

In [20]:
#remove links, numbers and any other simbols. Convert to lower case
def clean_up(s):
    s=re.sub(r'http\S+','',s)
    s=re.sub('\W', ' ',s)
    s=re.sub('\d', ' ', s)
    s=s.lower().strip()
    s=s.replace('á','a')
    s=s.replace('é','e')
    s=s.replace('í','i')
    s=s.replace('ó','o')
    s=s.replace('ú','u')
    return s

In [21]:
train_twitter['content']=train_twitter['content'].apply(lambda x : clean_up(x))

In [396]:
train_twitter['polarity'].value_counts()

0.0    757
1.0    471
0.5    130
Name: polarity, dtype: int64

In [23]:
#nlp = spacy.load('es_core_news_md')
#parser=Spanish()

In [ ]:
def clear_spaces(s):
    return [i for i in s if i != '   ' and i != '  ' and i != ' ' and i != '    ' and i != '     ']

In [23]:
train_twitter['lemmatized']=train_twitter['content'].apply(lambda x : lemmatize_(x))

In [24]:
prueba=train_twitter['lemmatized'][0]

In [26]:
train_twitter['tokens']=train_twitter['tokens'].apply(lambda x: clear_spaces(x))

In [27]:
train_twitter['lemmatized']=train_twitter['lemmatized'].apply(lambda x: clear_spaces(x))

# WORD CLOUD

In [29]:
#def remove_sw(s):
#    return [i for i in s if i not in STOP_WORDS and len(i)>2]

In [30]:
#train_twitter['tokens']=train_twitter['tokens'].apply(lambda x: remove_sw(x))
#train_twitter['lemmatized']=train_twitter['lemmatized'].apply(lambda x: remove_sw(x))

In [31]:
#wc=WordCloud().generate(','.join(train_twitter['tokens'][0]))

In [32]:
#plt.imshow(wc)

# TdIdf

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.probability import FreqDist

In [25]:
vect = TfidfVectorizer(stop_words = STOP_WORDS).fit(train_twitter.content)

In [26]:
x = vect.transform(train_twitter.content)

In [27]:
train_vectorized = pd.DataFrame(x.toarray(), columns = vect.get_feature_names())

In [28]:
train_vectorized=pd.concat([train_vectorized, train_twitter['polarity']], axis=1)

In [29]:
train_vectorized['polarity']=train_vectorized['polarity'].astype(int)

In [397]:
train_vectorized

,__walter__,_alexkun,_alineepa,_carlosrivera,_esme,_nosoyartista,_zylx,aaaa,aaaaajajajajajajajajaj,aaah,...,yyyy,zahirecoronel,zavala,ziempre,zommix,zona,zone,ñammm,ñoña,polarity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [30]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [31]:
import nltk

In [32]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB, ComplementNB
from sklearn.svm import SVC

# LOGISTIC REGRESSION

In [35]:
# Define X and y
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

# Train
log_reg = LogisticRegression().fit(X_train,y_train)

# Predict the labels
y_predicted = log_reg.predict(X_test)

# Print accuracy score and confusion matrix on test set
print('Accuracy on the test set: ', accuracy_score(y_test,y_predicted))
print(confusion_matrix(y_test,y_predicted)/len(y_test))

Accuracy on the test set:  0.7095588235294118
[[0.66911765 0.00367647]
 [0.28676471 0.04044118]]


# GAUSSIAN NB

In [36]:
# Define X and y
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.6213235294117647
[[0.42279412 0.25      ]
 [0.12867647 0.19852941]]


# MULTINOMIAL NB

In [37]:
# Define X and y
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

mnb=MultinomialNB()

y_pred = mnb.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.7279411764705882
[[0.65441176 0.01838235]
 [0.25367647 0.07352941]]


# BernoulliNB

In [38]:
# Define X and y
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

bnb=BernoulliNB()

y_pred = bnb.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.6838235294117647
[[0.65808824 0.01470588]
 [0.30147059 0.02573529]]


# SVC

In [39]:
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

svc=SVC()

y_pred = svc.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.7169117647058824
[[0.66911765 0.00367647]
 [0.27941176 0.04779412]]


# ComplementNB

In [40]:
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

cnb=ComplementNB()

y_pred = cnb.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.6727941176470589
[[0.4375     0.23529412]
 [0.09191176 0.23529412]]


In [41]:
from sklearn.ensemble import GradientBoostingClassifier as GBC

# GBC

In [42]:
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

gbc=GBC()

y_pred = gbc.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.7279411764705882
[[0.63970588 0.03308824]
 [0.23897059 0.08823529]]


In [46]:
from xgboost import XGBClassifier as XGBC

# XGBC

In [47]:
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

xgbc=XGBC()

y_pred = xgbc.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.7242647058823529
[[0.60661765 0.06617647]
 [0.20955882 0.11764706]]


# RANDOM FOREST CLASSIFIER

In [48]:
from sklearn.ensemble import RandomForestClassifier as RFC

In [49]:
y = train_vectorized.polarity
X = train_vectorized.drop('polarity', axis=1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=46)

rfc=RFC()

y_pred = rfc.fit(X_train, y_train).predict(X_test)
print('Accuracy on the test set: ', accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred)/len(y_test))

Accuracy on the test set:  0.7316176470588235
[[0.63970588 0.03308824]
 [0.23529412 0.09191176]]


# MODEL PREDICTION

In [362]:
import pickle

In [363]:
Pkl_Filename = "Pickle_RFC_Model.pkl"

In [364]:
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(rfc, file)

In [365]:
with open(Pkl_Filename, 'rb') as file:  
    Pickled_RFC_Model = pickle.load(file)

Pickled_RFC_Model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [374]:
Y_predict = Pickled_RFC_Model.predict(XX_test)

In [297]:
import h2o
from h2o.automl import H2OAutoML

In [298]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.7+8-LTS, mixed mode)
  Starting server from C:\Users\danie\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpfeujc7so
  JVM stdout: C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpfeujc7so\h2o_danie_started_from_python.out
  JVM stderr: C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpfeujc7so\h2o_danie_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_danie_ocslln
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.971 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"accepting new members, healthy"


In [300]:
train_vectorized.to_csv('trainh20.csv')

In [301]:
train=h2o.import_file('trainh20.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [302]:
X=train.columns
y='polarity'
X.remove(y)

In [304]:
aml=H2OAutoML(max_models=10, seed=1)
aml.train(x=X, y=y, training_frame=train)

AutoML progress: |
17:19:21.238: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [305]:
lb=aml.leaderboard

In [306]:
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200516_171921,0.175016,0.418349,0.175016,0.35537,0.29333
StackedEnsemble_AllModels_AutoML_20200516_171921,0.175211,0.418583,0.175211,0.356024,0.293454
GBM_2_AutoML_20200516_171921,0.184019,0.428975,0.184019,0.375741,0.300958
GBM_4_AutoML_20200516_171921,0.184747,0.429822,0.184747,0.371523,0.301468
GBM_3_AutoML_20200516_171921,0.185442,0.43063,0.185442,0.375282,0.301968
GBM_1_AutoML_20200516_171921,0.187915,0.433492,0.187915,0.383834,0.304471
GBM_grid__1_AutoML_20200516_171921_model_1,0.188263,0.433893,0.188263,0.390407,0.304303
XRT_1_AutoML_20200516_171921,0.189461,0.435272,0.189461,0.357056,0.303179
DRF_1_AutoML_20200516_171921,0.190027,0.435921,0.190027,0.357961,0.303662
GLM_1_AutoML_20200516_171921,0.210235,0.458514,0.210235,0.433346,0.321974


In [314]:
X_test.to_csv('testh20.csv')

In [315]:
test=h2o.import_file('testh20.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
